<a href="https://colab.research.google.com/github/Maplemx/Agently/blob/main/playground/workflow_series_02_using_condition_to_branch_off_inbpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Condition to Branch Off `#Agently_Workflow_Showcase_Series - 02`

## Demo Description

**Author:** Agently Team

**Series Link**:

[Last Document]: [01 - Let's Get Started by Building a Multi-Round Chat with Agently Workflow!](https://github.com/Maplemx/Agently/blob/main/playground/workflow_series_01_building_a_multi_round_chat.ipynb)

[Next Document]: [03 - Using Decorator to Create Chunks Faster](https://github.com/Maplemx/Agently/blob/main/playground/workflow_series_03_using_decorator_to_create_chunks.ipynb)

**Description:**

In the first showcase, we demostrated how to use **Agently Workflow** to create a multi-round chat.

You may notice that user can not exit the chat loop unless using the interupt command. That'll cause ugly error report and of course we can not use that in product.

So, in this showcase, let me introduce the **condition** feature of Agently Workflow. We can use this feature to make a judgement according the data value from the output handle and branch off the connection to different input handles. Then we can put an end to the infinite chat loop.

在第一个案例中，我们演示了如何使用框架v3.2版本**Agently Workflow**的新功能来创建一个多轮对话。你可能也注意到了，在上一个演示中，我们的用户除了使用强制终止按钮之外，没有别的办法可以退出多轮对话。而点击强制终止按钮，则会产生一堆报错信息，并且，我们也不可能在生产环境中让用户进行强制终止操作。

所以，在这一个案例中，我们将介绍**condition**这个能力。我们可以使用这个能力对输出端点(output handle)输出的数据值进行判断，并根据判断结果，将输出端点连接到不同的下游输入端点（input handle）去，从而实现让用户能够退出这个无限的循环。

## Step 1: Install Packages

In [ ]:
!pip install -q -U Agently

> ℹ️ Agently Workflow is a new feature in version >= `3.2.0`

## Step 2: Create a Multi-Round Chat Workflow in the Basic Way

### Workflow Graph

**Workflow in the Original Showcase:**

<img width = "640" src = "https://github.com/Maplemx/Agently/assets/4413155/6da2539d-afd8-4b57-9af8-0135e6afd61d"></img>

**Optimized Workflow in This Showcase:**

<img width = "640" src = "https://github.com/Maplemx/Agently/assets/4413155/2be63331-c119-4c93-aa14-fc138004976d"></img>

### Code

In [15]:
# SAME CODE FROM THE FIRST SHOWCASE
# FROM HERE =======================
import Agently
import logging

agent_factory = (
        Agently.AgentFactory()
        .set_settings("model.Google.auth.api_key", "")
        .set_settings("current_model", "Google")
)

# Step 0. Create Workflow Instance and Agent Instance
workflow = Agently.Workflow()
# reset logger level to WARNING
workflow.settings.set("logger", logging.getLogger("Workflow").setLevel(logging.WARNING))
agent = agent_factory.create_agent()

# Step 1. Create Chunks
## Start Chunk
start_chunk = workflow.schema.create_chunk(
    title = "Multi-Round Chat",
    type = "Start"
)

## User Input Chunk
user_input_chunk = workflow.schema.create_chunk(
    title = "User Input",
    handles = {
        "outputs": [{ "handle": "user_input" }],
    },
    executor = lambda inputs, storage: {
        "user_input": input("[User]: ")
    },
)

## Assistant Reply Chunk
def assistant_reply_executor(inputs, storage):
    chat_history = storage.get("chat_history") or []
    reply = (
        agent
            .chat_history(chat_history)
            .input(inputs["user_input"])
            .start()
    )
    print("[Assistant]: ", reply)
    return { "assistant_reply": reply }
assistant_reply_chunk = workflow.schema.create_chunk(
    title = "Assistant Reply",
    handles = {
        "inputs": [{ "handle": "user_input" }],
        "outputs": [{ "handle": "assistant_reply" }],
    },
    executor = assistant_reply_executor,
)

## Update Chat History Chunk
def update_chat_history_executor(inputs, storage):
    chat_history = storage.get("chat_history") or []
    chat_history.append({ "role": "user", "content": inputs["user_input"] })
    chat_history.append({ "role": "assistant", "content": inputs["assistant_reply"] })
    storage.set("chat_history", chat_history)
    return
update_chat_history_chunk = workflow.schema.create_chunk(
    title = "Update Chat History",
    handles = {
        "inputs": [
            { "handle": "user_input" },
            { "handle": "assistant_reply" },
        ],
    },
    executor = update_chat_history_executor,
)

# ======================= TO HERE

## Add a new chunk named `Goodbye Chunk`
goodbye_chunk = workflow.schema.create_chunk(
    title = "Goodbye Chunk",
    executor = lambda inputs, storage: print("Bye~👋")
)

# Step 2. Connect Chunks in Orders
start_chunk.connect_to(user_input_chunk)
# Let's change the output handle of `user_input_chunk`
# FROM:
# user_input_chunk.handle("user_input").connect_to(assistant_reply_chunk.handle("user_input"))
# TO:
(
    user_input_chunk.handle("user_input")
        .if_condition(lambda data: data == "#exit").connect_to(goodbye_chunk)
        .else_condition().connect_to(assistant_reply_chunk.handle("user_input"))
)
user_input_chunk.handle("user_input").connect_to(update_chat_history_chunk.handle("user_input"))
assistant_reply_chunk.handle("assistant_reply").connect_to(update_chat_history_chunk.handle("assistant_reply"))
update_chat_history_chunk.connect_to(user_input_chunk)

# Step 3. Start Workflow
workflow.start()

User Input
User Input
[User]: Hi, I'm Max
[Assistant]:  Hello Max, nice to meet you.
[User]: What's your name?
[Assistant]:  I am Gemini, a multimodal AI language model developed by Google.
[User]: #exit
Bye~👋


---

[**_<font color = "red">Agent</font><font color = "blue">ly</font>_** Framework - Speed up your AI Agent Native application development](https://github.com/Maplemx/Agently)